# Tarea 5

**Envío del PR inicial:** 15 de octubre

**Aceptación del PR:** 21 de octubre

NOTA: Esta tarea debe entregarse en equipo.

In [1]:
include("Mapeos.jl")
include("AutoDiff.jl")

┌ Warning: Deprecated syntax `(u.x′ >= 0) ? print(io, u.x, "+", u.x′, "\u03b5"):` at /home/uzielnmtz/TSFisComp1/tareas/UzielLinares/AutoDiff.jl:18.
│ Use `(u.x′ >= 0) ? print(io, u.x, "+", u.x′, "\u03b5") :` instead.
└ @ nothing /home/uzielnmtz/TSFisComp1/tareas/UzielLinares/AutoDiff.jl:18


Main.AutoDiff

In [2]:
using .Mapeos
using .AutoDiff

In [3]:
function newton_dual(f, guess; n_iter=1000)
    xnn = dual(guess)
    for i in 1:n_iter
        xnn = xnn.x - (f(xnn).x / f(xnn).x′)
        xnn = dual(xnn)
    end
    return xnn.x
end

newton_dual (generic function with 1 method)

In [4]:
Qc(x, c) = x^2 + c

Qc (generic function with 1 method)

In [5]:
fₙ(cn) = (cn[1] - cn[2]) / (cn[2] - cn[3])

fₙ (generic function with 1 method)

In [6]:
function compone(f, x, n)
    n_comp = x
    for i in 1:n
        n_comp = f(n_comp)
    end
    return n_comp
end

compone (generic function with 1 method)

## 1

Llamemos $c_n$ el valor del parámetro $c$ donde ocurre la bifurcación de doblamiento de periodo para el mapeo $Q_c(x)=x^2+c$, donde la órbita de periodo $2^n$ nace. Como hemos visto en notebooks anteriores, tenemos que $c_0=1/4$ marca la aparición del atractor de periodo $2^0=1$, $c_1=-1/4$ corresponde a la aparición del atractor de periodo $2^1=2$, $c_2=-3/4$ a la aparición del atractor de periodo $2^2=4$, etc. 

A partir de estos valores y otros que calcularán (al menos deben encontrar $c_6$), definimos la secuencia: $\{f_0, f_1, f_2, \dots\}$, donde

\begin{equation}
f_n = \frac{c_n-c_{n+1}}{c_{n+1}-c_{n+2}} .
\end{equation}

La pregunta es, ¿a qué valor converge esta secuencia?, es decir, dar una estimación de $f_\infty$.


*Hint:* Para realizar este ejercicio deben calcular el atractor para varios valores de $c$, de tal manera que puedan aislar las órbitas de periodo $2^p$ y de ahí determinar varios valores $c_n$. Se requerir suficiente cuidado para obtener una buena aproximación de $c_n$. 
Una opción se basa en recordar/usar que las bifurcaciones de doblamiento de periodo ocurren cuando los puntos de la órbita de periodo $p$ se tornan en repulsores, es decir, $(Q_c^p)'(x)=-1$. Esta opción, entonces, involucra obtener los valores $c_n$ a partir de órbitas periódicas de periodo $2^n$ usando los polinomios $Q_c^{2^p}(x)$ y diferenciación automática.

In [7]:
"""
    calcula_cₙ(Fλ, rango_c, n; ϵ=0.01, n_iter=1000)

Calcula el valor aproximado de 'c_n'

"""

function calcula_cₙ(Fλ, rango_c, n; ϵ=0.01, n_iter=1000)
    cn = []
    deltas = []
    ders = []
    x0 = 0.
    for c in rango_c
        F = x -> Fλ(x, c)
        m = Mapeo(F, x0, n_iter)
        atr = dual(m.xnn[end])
        der = compone(F, atr, Int(2^n)).x′
        #println(der)
        δ = abs(der + 1)
        if δ < ϵ
            push!(cn, c)
            push!(ders, der)
            push!(deltas, δ)
        end
        x0 = atr.x
    end
    if !isempty(deltas)
        _, ind = findmin(deltas)
        return cn[ind]
    else
        return Nothing
    end
end

┌ Warning: Deprecated syntax `multiple line breaks between doc string and object`.
│ Use `at most one line break` instead.
└ @ nothing none:8


calcula_cₙ

El anterior codigo busca la bifuracion de periodo $2^n$ en el intervalo `rango_c`, entonces busquemos uno a uno estos puntos.

Por completez, encontremos veamos que estyo encuentra al primer punto de bifuracion de doblamiento de periodo $c_n=-3/4$.

In [8]:
rango_c = reverse(-1.5:1e-2:-1/2.)

-0.5:-0.01:-1.5

In [9]:
c_1 = calcula_cₙ(Qc, rango_c, 0)

-0.75

Continuemos la busqueda tomando intervalos cada vez mas chicos en el cual uno de los extremos es la $c_n$ encontrada previamente.

In [10]:
rango_c = reverse(-1.5:1e-3:c_1)
c_2 = calcula_cₙ(Qc, rango_c, 1)

-1.25

In [11]:
rango_c = reverse(-1.4:1e-4:c_2)
c_3 = calcula_cₙ(Qc, rango_c, 2)

-1.3681

In [12]:
rango_c = reverse(-1.4:1e-4:c_3)
c_4 = calcula_cₙ(Qc, rango_c, 3)

-1.394

In [13]:
rango_c = reverse(-1.5:1e-4:c_4)
c_5 = calcula_cₙ(Qc, rango_c, 4)

-1.3996

In [14]:
rango_c = reverse(-1.5:1e-5:c_5)
c_6 = calcula_cₙ(Qc, rango_c, 5)

-1.40083

In [15]:
rango_c = reverse(-1.5:1e-6:c_6)
c_7 = calcula_cₙ(Qc, rango_c, 6)

-1.446791

In [16]:
# Busquemos este ultimo con mas fineza
rango_c = reverse((c_6-1e-3):1e-8:c_6)
c_7 = calcula_cₙ(Qc, rango_c, 6)

-1.40108525

Entonces, los valores encontrados son los siguientes.

In [17]:
cn = [c_1, c_2, c_3, c_4, c_5, c_6, c_7]

7-element Array{Float64,1}:
 -0.75      
 -1.25      
 -1.3681    
 -1.394     
 -1.3996    
 -1.40083   
 -1.40108525

In [18]:
for (i, c) in enumerate(cn)
    println(i, "  ", c)
end

1  -0.75
2  -1.25
3  -1.3681
4  -1.394
5  -1.3996
6  -1.40083
7  -1.40108525


In [84]:
fₙ(cn[end-2:end])

4.818805093047623

## 2

Repitan el ejercicio anterior para el mapeo $S_c(x) = c \sin(x)$. 

- ¿Cómo se comparan los valores obtenidos de $f_n$? 

- ¿Qué interpretación le pueden dar a este resultado, en comparación del ejercicio anterior?

In [20]:
Sc(x, c) = c * sin(x)

Sc (generic function with 1 method)

## 3

Como se ve en la Fig. 1 del diagrama de bifurcaciones de $Q_c$, $x=0$ pertenece a un ciclo de periodo $2^n$ para ciertos valores $C_n$ del parámetro. Dichos valores son *especiales*, ya que el hecho de que $x=0$ pertenezca a un ciclo de periodo $2^n$ define los llamados *ciclos superestable*, donde tenemos $(Q^{2^p}_{C_n})'(0)=0$.

- ¿A qué converge la secuencia $f_n$, definida ahora a partir de los valores $C_n$.

- De los $2^p$ puntos del ciclo de periodo $2^p$, es decir, $\{0, p_1, \dots p_{2^{n-1}}\,\}$ hay uno (distinto del 0) cuya distancia a 0 es la menor; a esa distancia la identificaremos como $d_n$. Calcular numéricamente a qué converge la secuencia $d_n/d_{n+1}$.

Para esto primero generemos distintas trayectorias y veamos quienes tienen al $0$ como punto fijo.
Puesto que se van a tomar los ultimos puntos en las trayectorias se asume que se ha alcanzado su comportamiento "estable", por lo cual bastara encontrar a las trayectorias que tengan 0 en alguno de sus puntos.

In [8]:
rango_c = -5/4:0.0001:1
Fs = [x -> Qc(x, c) for c in rango_c];

x₀ = 0.0
mapeos = [Mapeo(F, x₀, 1000) for F in Fs];

# se va a tomar los últimos 101 puntos de cada mapeo
ps = [m.xnn[end-100:end] for m in mapeos];


Cn1 = []
δ = 0.00001
for (i, c) in enumerate(rango_c)
    if any(abs.(ps[i]) .< δ)
        push!(Cn1, c)
    end
end

In [9]:
Cn1

2-element Array{Any,1}:
 -1.0
  0.0

In [10]:
rango_c = -1.5:0.00001:-5/4
Fs = [x -> Qc(x, c) for c in rango_c];

x₀ = 0.0
mapeos = [Mapeo(F, x₀, 1_000) for F in Fs];

# se va a tomar los últimos 101 puntos de cada mapeo
ps = [m.xnn[end-100:end] for m in mapeos];




Cn2 = []
δ = 0.00003
for (i, c) in enumerate(rango_c)
    if any(abs.(ps[i]) .< δ)
        push!(Cn2, c)
    end
end


In [11]:
Cn2

38-element Array{Any,1}:
 -1.49809
 -1.49697
 -1.4964 
 -1.49597
 -1.48719
 -1.4857 
 -1.48534
 -1.48483
 -1.48465
 -1.48351
 -1.46685
 -1.45818
 -1.45779
  ⋮      
 -1.40787
 -1.40643
 -1.40332
 -1.40307
 -1.40241
 -1.40132
 -1.39695
 -1.38155
 -1.38154
 -1.31071
 -1.3107 
 -1.31069

In [12]:
Cn = reverse(vcat(Cn2, Cn1))

40-element Array{Any,1}:
  0.0    
 -1.0    
 -1.31069
 -1.3107 
 -1.31071
 -1.38154
 -1.38155
 -1.39695
 -1.40132
 -1.40241
 -1.40307
 -1.40332
 -1.40643
  ⋮      
 -1.45818
 -1.46685
 -1.48351
 -1.48465
 -1.48483
 -1.48534
 -1.4857 
 -1.48719
 -1.49597
 -1.4964 
 -1.49697
 -1.49809

Esta ultima lista contiene los valores de $C_n$, notemos que por errores de precision hay varios puntos que se refieren al mismo punto, por eso creemos la siguiente lista en la cual se incluyen los primeros 5 puntos de interes.

In [81]:
Cn = [0.0, -1.0, -1.31069, -1.38154, -1.39695]

5-element Array{Float64,1}:
  0.0    
 -1.0    
 -1.31069
 -1.38154
 -1.39695

Entonces

In [82]:
fₙ(Cn[end-2:end])

4.5976638546398725

Del primer problema tenemos que las primeras bifurcaciones se dan en las siguientes $c$.

In [83]:
println("Periodo\tC")
for (i, c) in enumerate(cn)
    println(2^(i),"\t",c)
end

Periodo	C
2	-0.75
4	-1.25
8	-1.3681
16	-1.394
32	-1.3996
64	-1.40083
128	-1.40108525


Generemos trayectorias para estas $c$

In [107]:
Fs = [x -> Qc(x, c-1e-4) for c in cn[1:end]] # el valor sumado a la c es para garantizar que nos encontramos en ese periodo
mapeos = [Mapeo(F, 0, 100_000) for F in Fs];
ps = [m.xnn[end-100:end] for m in mapeos];

In [108]:
dd = []
δ = 0.00001
for p in ps
    aux = [val for val in p if δ < abs(val)]
    d = minimum(abs.(aux))
    push!(dd, d)
end

In [109]:
dd

7-element Array{Any,1}:
 0.4899999999894546   
 0.19883749258369887  
 0.07652091680610718  
 0.02900717902772909  
 0.009001045486740189 
 0.0006989396045056218
 0.0035559287663733397

In [110]:
for i in 1:length(dd)-1
    println("n=$i", "\t", dd[i] / dd[i+1])
end

n=1	2.4643239744294876
n=2	2.5984724292773964
n=3	2.637999259871422
n=4	3.2226455327284773
n=5	12.878144876490243
n=6	0.19655613214616335
